In [32]:
import os
import glob

from IPython.display import display, Markdown
import pandas as pd

# Group types of models (experiment type and model type) and pick best performing in terms of f1-score per unit of analysis and report them in a table


In [33]:
results_filepaths_mean_pred = glob.glob('./logged_performance_per_model/*/*raw_mean*.csv')
results_filepaths_majority_pred = glob.glob('./logged_performance_per_model/*/*raw_majority*.csv')

In [34]:
def conantenate_results(filepath_list):
    dfs_list = []
    for results_filepath in filepath_list:
        model_name = results_filepath.split('/')[-2]
        results_df_i = pd.read_csv(results_filepath)
        results_df_i['model_name'] = model_name
        dfs_list.append(results_df_i)

    results_df_ = pd.concat(dfs_list).set_index(['language', 'model_name', 'unit_of_analysis']).sort_index()
    results_df_.rename(columns={'f1-mico_mean': 'f1-micro_mean', 'f1-mico_std': 'f1-micro_std'}, inplace=True)

    return results_df_

In [35]:
results_mean_pred_df = conantenate_results(results_filepaths_mean_pred)
results_majority_vote_pred_df = conantenate_results(results_filepaths_majority_pred)

### Generate the tables to report

In [36]:
def display_performance_table(df, metric, index_cols=['model_name'], display_=True):
    report_table = df.reset_index().copy()
    report_table['result'] = report_table[f'{metric}'].map(lambda x: f'{x:.3f}')
    report_table['col_title'] = report_table.unit_of_analysis.str.split('_').str.join(' ') 
    report_table['col_title'] = pd.Categorical(
        report_table.col_title,
        categories=['title', 'title and first paragraph', 'title and 5 sentences', 'title and 10 sentences',
                    'title and first sentence each paragraph', 'raw text'],
        ordered=True)
    report_table = report_table[index_cols + ['col_title', 'result']]\
        .pivot_table(index=index_cols, columns=['col_title'], values=['result'], aggfunc='first', fill_value=0)\
        .droplevel(0, axis=1)

    report_table.columns.names = [None]

    # Highlight best scoring models according to their average
    mean_perf_arr = report_table.applymap(lambda x: float(str(x).split(' ')[0])).to_numpy()
    highlight_mask = mean_perf_arr == mean_perf_arr.max()
    report_table_arr = report_table.to_numpy()  # Note it passes the array by reference
    report_table_arr[highlight_mask] = '**' + report_table_arr[highlight_mask] + '**'

    if display_:
        display(Markdown(report_table.to_markdown()))
    
    return report_table

### Generate tables for all languages

In [37]:
metrics_to_report = ['f1_micro', 'recall_micro', 'precision_micro', 'accuracy']

In [38]:
language_dict = {'en': 'English', 'it': 'Italian', 'fr': 'French', 'po': 'Polish', 'ru': 'Russian', 'ge': 'German'}

In [39]:
def display_metrics_and_write_to_file(df, grouping_criterion, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    report_tables_dfs_dict = {metric: [] for metric in metrics_to_report}

    for language, results_df in df.groupby(level=0):
        display(Markdown(f'# {language_dict[language]}'))
        
        for metric in metrics_to_report:
            os.makedirs(os.path.join(output_dir, metric), exist_ok=True)

            output_dir_markdown = os.path.join(output_dir, metric, 'markdown')
            output_dir_latex = os.path.join(output_dir, metric, 'latex')
            output_dir_csv = os.path.join(output_dir, metric, 'csv')

            os.makedirs(output_dir_markdown, exist_ok=True)
            os.makedirs(output_dir_latex, exist_ok=True)
            os.makedirs(output_dir_csv, exist_ok=True)

            display(Markdown(f'## {metric}'))

            report_table = display_performance_table(df=results_df, index_cols=grouping_criterion, metric=metric, display_=True)

            # Export as markdown
            markdown_file = open(os.path.join(output_dir_markdown, f"{language_dict[language]}_{metric}.md"), "w")
            report_table.reset_index().to_markdown(markdown_file, index=False)
            markdown_file.close()

            # Export as latex table
            latex_file = open(os.path.join(output_dir_latex, f"{language_dict[language]}_{metric}.tex"), "w")
            report_table.reset_index().to_latex(latex_file, index=False)
            latex_file.close()

            # Export as csv
            report_table.to_csv(os.path.join(output_dir_csv, f"{language_dict[language]}_{metric}.csv"))

            # Stack all languages into single table
            report_table['language'] = language
            report_table = report_table.reset_index().set_index(['language'] + grouping_criterion)

            report_tables_dfs_dict[metric].append(report_table)

    # Report or store unified table
    display(Markdown(f'# All 6 Languages'))
    for metric in metrics_to_report:
        display(Markdown(f'## {metric}'))
        multi_language_report_table_metric = pd.concat(report_tables_dfs_dict[metric])
        display(Markdown(multi_language_report_table_metric.reset_index().to_markdown(index=False)))

        output_dir_markdown = os.path.join(output_dir, metric, 'markdown')
        output_dir_latex = os.path.join(output_dir, metric, 'latex')
        output_dir_csv = os.path.join(output_dir, metric, 'csv')

        # Export as markdown
        markdown_file = open(os.path.join(output_dir_markdown, f"all_6_languages_{metric}.md"), "w")
        multi_language_report_table_metric.reset_index().to_markdown(markdown_file, index=False)
        markdown_file.close()

        # Export as latex table
        latex_file = open(os.path.join(output_dir_latex, f"all_6_languages_{metric}.tex"), "w")
        multi_language_report_table_metric.reset_index().to_latex(latex_file, index=False)
        latex_file.close()

        # Export as csv
        multi_language_report_table_metric.to_csv(os.path.join(output_dir_csv, f"all_6_languages_{metric}.csv"))

# Per model type

In [40]:
display_metrics_and_write_to_file(df=results_mean_pred_df, grouping_criterion=['model_name'], output_dir='per_model_name_tables_mean_prediction')

# English

## f1_micro

| model_name                                 |   title |   title and first paragraph |   title and 5 sentences | title and 10 sentences   | title and first sentence each paragraph   |   raw text |
|:-------------------------------------------|--------:|----------------------------:|------------------------:|:-------------------------|:------------------------------------------|-----------:|
| AshtonIsNotHere-xlm-roberta-long-base-4096 |   0.64  |                       0.7   |                   0.684 | 0.699                    | 0.704                                     |      0.691 |
| EleutherAI-gpt-neo-1.3B                    |   0.63  |                       0.703 |                   0.707 | 0.693                    | 0.695                                     |      0.698 |
| EleutherAI-gpt-neo-125M                    |   0.53  |                       0.623 |                   0.637 | 0.634                    | 0.663                                     |      0.671 |
| bert-base-multilingual-cased               |   0.61  |                       0.677 |                   0.7   | 0.707                    | 0.684                                     |      0.693 |
| distilbert-base-multilingual-cased         |   0.596 |                       0.665 |                   0.661 | 0.672                    | 0.672                                     |      0.69  |
| facebook-mbart-large-50                    |   0.677 |                       0.717 |                   0.719 | **0.721**                | **0.721**                                 |      0.707 |
| gpt2                                       |   0.618 |                       0.693 |                   0.69  | 0.673                    | 0.692                                     |      0.696 |
| xlm-roberta-large                          |   0.66  |                       0.694 |                   0.718 | 0.715                    | 0.717                                     |      0.704 |

/tmp/ipykernel_25445/2331756451.py:30: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  report_table.reset_index().to_latex(latex_file, index=False)


## recall_micro

...

| language   | model_name                                 |   title |   title and first paragraph | title and 5 sentences   | title and 10 sentences   | title and first sentence each paragraph   |   raw text |
|:-----------|:-------------------------------------------|--------:|----------------------------:|:------------------------|:-------------------------|:------------------------------------------|-----------:|
| en         | AshtonIsNotHere-xlm-roberta-long-base-4096 |   0.717 |                       0.774 | 0.776                   | 0.765                    | 0.744                                     |      0.734 |
| en         | EleutherAI-gpt-neo-1.3B                    |   0.761 |                       0.794 | 0.824                   | 0.812                    | 0.793                                     |      0.778 |
| en         | EleutherAI-gpt-neo-125M                    |   0.646 |                       0.73  | 0.735                   | 0.752                    | 0.770                                     |      0.763 |
| en         | bert-base-multilingual-cased               |   0.696 |                       0.765 | 0.787                   | 0.776                    | 0.751                                     |      0.736 |
| en         | distilbert-base-multilingual-cased         |   0.684 |                       0.759 | 0.769                   | 0.735                    | 0.750                                     |      0.728 |
| en         | facebook-mbart-large-50                    |   0.778 |                       0.795 | 0.812                   | 0.804                    | 0.784                                     |      0.752 |
| en         | gpt2                                       |   0.709 |                       0.751 | 0.725                   | 0.711                    | 0.736                                     |      0.746 |
| en         | xlm-roberta-large                          |   0.759 |                       0.765 | **0.825**               | 0.804                    | 0.781                                     |      0.765 |
| fr         | AshtonIsNotHere-xlm-roberta-long-base-4096 |   0.455 |                       0.52  | 0.562                   | 0.505                    | 0.584                                     |      0.552 |
| fr         | EleutherAI-gpt-neo-1.3B                    |   0.536 |                       0.6   | 0.540                   | 0.542                    | 0.578                                     |      0.581 |
| fr         | EleutherAI-gpt-neo-125M                    |   0.324 |                       0.463 | 0.480                   | 0.440                    | 0.524                                     |      0.57  |
| fr         | bert-base-multilingual-cased               |   0.524 |                       0.538 | 0.495                   | 0.532                    | 0.545                                     |      0.571 |
| fr         | distilbert-base-multilingual-cased         |   0.481 |                       0.51  | 0.520                   | 0.545                    | 0.610                                     |      0.53  |
| fr         | facebook-mbart-large-50                    |   0.558 |                       0.559 | **0.640**               | 0.570                    | 0.589                                     |      0.609 |
| fr         | gpt2                                       |   0.371 |                       0.443 | 0.483                   | 0.505                    | 0.512                                     |      0.549 |
| fr         | xlm-roberta-large                          |   0.505 |                       0.545 | 0.577                   | 0.547                    | 0.594                                     |      0.558 |
| ge         | AshtonIsNotHere-xlm-roberta-long-base-4096 |   0.702 |                       0.662 | 0.686                   | 0.773                    | 0.695                                     |      0.701 |
| ge         | EleutherAI-gpt-neo-1.3B                    |   0.678 |                       0.703 | 0.705                   | **0.802**                | 0.733                                     |      0.721 |
| ge         | EleutherAI-gpt-neo-125M                    |   0.533 |                       0.641 | 0.680                   | 0.651                    | 0.667                                     |      0.697 |
| ge         | bert-base-multilingual-cased               |   0.608 |                       0.677 | 0.698                   | 0.707                    | 0.628                                     |      0.673 |
| ge         | distilbert-base-multilingual-cased         |   0.658 |                       0.636 | 0.638                   | 0.699                    | 0.685                                     |      0.653 |
| ge         | facebook-mbart-large-50                    |   0.741 |                       0.667 | 0.672                   | 0.760                    | 0.704                                     |      0.701 |
| ge         | gpt2                                       |   0.559 |                       0.624 | 0.587                   | 0.652                    | 0.650                                     |      0.647 |
| ge         | xlm-roberta-large                          |   0.721 |                       0.705 | 0.711                   | 0.735                    | 0.723                                     |      0.719 |
| it         | AshtonIsNotHere-xlm-roberta-long-base-4096 |   0.705 |                       0.657 | 0.661                   | 0.735                    | 0.727                                     |      0.708 |
| it         | EleutherAI-gpt-neo-1.3B                    |   0.644 |                       0.731 | 0.756                   | 0.726                    | 0.713                                     |      0.716 |
| it         | EleutherAI-gpt-neo-125M                    |   0.392 |                       0.483 | 0.606                   | 0.586                    | 0.600                                     |      0.69  |
| it         | bert-base-multilingual-cased               |   0.569 |                       0.703 | 0.669                   | 0.692                    | 0.671                                     |      0.689 |
| it         | distilbert-base-multilingual-cased         |   0.57  |                       0.651 | 0.671                   | 0.669                    | 0.639                                     |      0.686 |
| it         | facebook-mbart-large-50                    |   0.678 |                       0.732 | 0.760                   | **0.780**                | 0.733                                     |      0.734 |
| it         | gpt2                                       |   0.487 |                       0.578 | 0.587                   | 0.608                    | 0.622                                     |      0.582 |
| it         | xlm-roberta-large                          |   0.673 |                       0.729 | 0.765                   | 0.764                    | 0.729                                     |      0.758 |
| po         | AshtonIsNotHere-xlm-roberta-long-base-4096 |   0.604 |                       0.661 | 0.682                   | 0.750                    | 0.762                                     |      0.711 |
| po         | EleutherAI-gpt-neo-1.3B                    |   0.566 |                       0.659 | 0.725                   | 0.750                    | 0.725                                     |      0.713 |
| po         | EleutherAI-gpt-neo-125M                    |   0.544 |                       0.591 | 0.586                   | 0.622                    | 0.647                                     |      0.664 |
| po         | bert-base-multilingual-cased               |   0.633 |                       0.69  | 0.733                   | 0.724                    | 0.755                                     |      0.693 |
| po         | distilbert-base-multilingual-cased         |   0.647 |                       0.592 | 0.679                   | 0.747                    | 0.709                                     |      0.656 |
| po         | facebook-mbart-large-50                    |   0.614 |                       0.679 | 0.770                   | **0.792**                | 0.786                                     |      0.738 |
| po         | gpt2                                       |   0.559 |                       0.611 | 0.586                   | 0.684                    | 0.660                                     |      0.694 |
| po         | xlm-roberta-large                          |   0.644 |                       0.703 | 0.711                   | 0.744                    | 0.782                                     |      0.753 |
| ru         | AshtonIsNotHere-xlm-roberta-long-base-4096 |   0.5   |                       0.522 | 0.545                   | 0.561                    | 0.633                                     |      0.583 |
| ru         | EleutherAI-gpt-neo-1.3B                    |   0.576 |                       0.515 | 0.639                   | 0.632                    | 0.643                                     |      0.571 |
| ru         | EleutherAI-gpt-neo-125M                    |   0.36  |                       0.341 | 0.385                   | 0.288                    | 0.258                                     |      0.265 |
| ru         | bert-base-multilingual-cased               |   0.464 |                       0.486 | 0.580                   | 0.537                    | 0.529                                     |      0.521 |
| ru         | distilbert-base-multilingual-cased         |   0.457 |                       0.434 | 0.556                   | 0.541                    | 0.529                                     |      0.484 |
| ru         | facebook-mbart-large-50                    |   0.58  |                       0.577 | 0.655                   | 0.614                    | **0.706**                                 |      0.588 |
| ru         | gpt2                                       |   0.333 |                       0.231 | 0.190                   | 0.263                    | 0.308                                     |      0.326 |
| ru         | xlm-roberta-large                          |   0.562 |                       0.585 | 0.607                   | 0.559                    | 0.594                                     |      0.524 |

/tmp/ipykernel_25445/2331756451.py:60: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  multi_language_report_table_metric.reset_index().to_latex(latex_file, index=False)


## accuracy

| language   | model_name                                 |   title | title and first paragraph   | title and 5 sentences   | title and 10 sentences   | title and first sentence each paragraph   | raw text   |
|:-----------|:-------------------------------------------|--------:|:----------------------------|:------------------------|:-------------------------|:------------------------------------------|:-----------|
| en         | AshtonIsNotHere-xlm-roberta-long-base-4096 |   0.087 | 0.136                       | 0.126                   | 0.068                    | 0.078                                     | 0.058      |
| en         | EleutherAI-gpt-neo-1.3B                    |   0.087 | 0.087                       | **0.155**               | 0.097                    | 0.117                                     | 0.087      |
| en         | EleutherAI-gpt-neo-125M                    |   0.019 | 0.087                       | 0.058                   | 0.039                    | 0.087                                     | 0.078      |
| en         | bert-base-multilingual-cased               |   0.078 | 0.087                       | 0.117                   | 0.078                    | 0.068                                     | 0.087      |
| en         | distilbert-base-multilingual-cased         |   0.097 | 0.078                       | 0.126                   | 0.078                    | 0.117                                     | 0.097      |
| en         | facebook-mbart-large-50                    |   0.117 | 0.126                       | 0.146                   | 0.107                    | 0.146                                     | 0.097      |
| en         | gpt2                                       |   0.097 | 0.097                       | 0.117                   | 0.087                    | 0.078                                     | 0.117      |
| en         | xlm-roberta-large                          |   0.058 | 0.087                       | 0.146                   | 0.126                    | 0.117                                     | 0.117      |
| fr         | AshtonIsNotHere-xlm-roberta-long-base-4096 |   0.071 | 0.071                       | 0.071                   | 0.071                    | **0.119**                                 | 0.071      |
| fr         | EleutherAI-gpt-neo-1.3B                    |   0.024 | **0.119**                   | 0.024                   | 0.071                    | 0.024                                     | 0.071      |
| fr         | EleutherAI-gpt-neo-125M                    |   0.024 | 0.024                       | 0.024                   | 0.024                    | 0.024                                     | 0.071      |
| fr         | bert-base-multilingual-cased               |   0.048 | 0.071                       | 0.048                   | 0.071                    | 0.071                                     | 0.095      |
| fr         | distilbert-base-multilingual-cased         |   0.071 | 0.071                       | 0.095                   | 0.048                    | 0.095                                     | 0.048      |
| fr         | facebook-mbart-large-50                    |   0.024 | 0.095                       | 0.071                   | 0.048                    | 0.071                                     | 0.071      |
| fr         | gpt2                                       |   0     | 0.024                       | 0.048                   | 0.000                    | 0.048                                     | 0.048      |
| fr         | xlm-roberta-large                          |   0     | **0.119**                   | 0.071                   | 0.095                    | 0.095                                     | 0.071      |
| ge         | AshtonIsNotHere-xlm-roberta-long-base-4096 |   0.029 | 0.029                       | 0.029                   | 0.057                    | 0.057                                     | 0.029      |
| ge         | EleutherAI-gpt-neo-1.3B                    |   0     | 0.029                       | 0.000                   | 0.057                    | 0.029                                     | 0.029      |
| ge         | EleutherAI-gpt-neo-125M                    |   0     | 0.000                       | 0.029                   | 0.000                    | 0.029                                     | 0.029      |
| ge         | bert-base-multilingual-cased               |   0     | 0.029                       | 0.029                   | 0.057                    | 0.000                                     | 0.029      |
| ge         | distilbert-base-multilingual-cased         |   0.029 | 0.029                       | 0.000                   | 0.000                    | 0.029                                     | 0.000      |
| ge         | facebook-mbart-large-50                    |   0.057 | 0.029                       | 0.029                   | 0.029                    | 0.057                                     | 0.000      |
| ge         | gpt2                                       |   0     | 0.000                       | 0.029                   | **0.086**                | 0.000                                     | 0.000      |
| ge         | xlm-roberta-large                          |   0.029 | 0.057                       | 0.029                   | 0.000                    | 0.057                                     | 0.029      |
| it         | AshtonIsNotHere-xlm-roberta-long-base-4096 |   0.1   | 0.150                       | 0.167                   | 0.133                    | 0.100                                     | 0.117      |
| it         | EleutherAI-gpt-neo-1.3B                    |   0.05  | 0.117                       | **0.183**               | 0.150                    | 0.133                                     | 0.133      |
| it         | EleutherAI-gpt-neo-125M                    |   0     | 0.000                       | 0.033                   | 0.067                    | 0.033                                     | 0.133      |
| it         | bert-base-multilingual-cased               |   0.017 | 0.117                       | 0.067                   | **0.183**                | 0.117                                     | 0.133      |
| it         | distilbert-base-multilingual-cased         |   0.017 | 0.117                       | 0.117                   | 0.150                    | 0.083                                     | 0.133      |
| it         | facebook-mbart-large-50                    |   0.1   | 0.167                       | 0.150                   | 0.167                    | 0.167                                     | **0.183**  |
| it         | gpt2                                       |   0.033 | 0.033                       | 0.033                   | 0.067                    | 0.083                                     | 0.050      |
| it         | xlm-roberta-large                          |   0.1   | 0.167                       | **0.183**               | 0.133                    | 0.150                                     | **0.183**  |
| po         | AshtonIsNotHere-xlm-roberta-long-base-4096 |   0.026 | 0.026                       | 0.026                   | 0.051                    | 0.051                                     | 0.051      |
| po         | EleutherAI-gpt-neo-1.3B                    |   0.026 | 0.000                       | 0.051                   | 0.051                    | 0.026                                     | 0.051      |
| po         | EleutherAI-gpt-neo-125M                    |   0     | 0.000                       | 0.051                   | 0.026                    | 0.026                                     | 0.026      |
| po         | bert-base-multilingual-cased               |   0.026 | 0.051                       | 0.051                   | 0.051                    | 0.051                                     | 0.051      |
| po         | distilbert-base-multilingual-cased         |   0.051 | 0.000                       | 0.026                   | 0.051                    | 0.051                                     | 0.026      |
| po         | facebook-mbart-large-50                    |   0.026 | 0.000                       | 0.026                   | **0.077**                | 0.026                                     | 0.051      |
| po         | gpt2                                       |   0     | 0.000                       | 0.000                   | 0.026                    | 0.026                                     | 0.051      |
| po         | xlm-roberta-large                          |   0.051 | 0.026                       | 0.026                   | 0.051                    | 0.026                                     | **0.077**  |
| ru         | AshtonIsNotHere-xlm-roberta-long-base-4096 |   0.158 | 0.158                       | 0.132                   | 0.079                    | **0.263**                                 | 0.237      |
| ru         | EleutherAI-gpt-neo-1.3B                    |   0.053 | 0.105                       | 0.079                   | 0.158                    | 0.105                                     | 0.105      |
| ru         | EleutherAI-gpt-neo-125M                    |   0.026 | 0.000                       | 0.053                   | 0.000                    | 0.026                                     | 0.026      |
| ru         | bert-base-multilingual-cased               |   0.079 | 0.105                       | 0.237                   | 0.158                    | 0.184                                     | 0.237      |
| ru         | distilbert-base-multilingual-cased         |   0.105 | 0.132                       | 0.158                   | 0.158                    | 0.211                                     | 0.158      |
| ru         | facebook-mbart-large-50                    |   0.158 | 0.211                       | 0.211                   | 0.184                    | 0.211                                     | 0.184      |
| ru         | gpt2                                       |   0     | 0.000                       | 0.000                   | 0.026                    | 0.053                                     | 0.026      |
| ru         | xlm-roberta-large                          |   0.158 | 0.184                       | 0.211                   | 0.105                    | 0.211                                     | 0.158      |

/tmp/ipykernel_25445/2331756451.py:60: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  multi_language_report_table_metric.reset_index().to_latex(latex_file, index=False)


In [41]:
display_metrics_and_write_to_file(df=results_majority_vote_pred_df, grouping_criterion=['model_name'], output_dir='per_model_name_tables_majority_voting')

# English

## f1_micro

| model_name                                 |   title |   title and first paragraph |   title and 5 sentences |   title and 10 sentences | title and first sentence each paragraph   |   raw text |
|:-------------------------------------------|--------:|----------------------------:|------------------------:|-------------------------:|:------------------------------------------|-----------:|
| AshtonIsNotHere-xlm-roberta-long-base-4096 |   0.64  |                       0.7   |                   0.684 |                    0.699 | 0.696                                     |      0.679 |
| EleutherAI-gpt-neo-1.3B                    |   0.63  |                       0.703 |                   0.707 |                    0.693 | 0.692                                     |      0.692 |
| EleutherAI-gpt-neo-125M                    |   0.53  |                       0.623 |                   0.637 |                    0.634 | 0.649                                     |      0.654 |
| bert-base-multilingual-cased               |   0.61  |                       0.677 |                   0.7   |                    0.707 | 0.682                                     |      0.688 |
| distilbert-base-multilingual-cased         |   0.596 |                       0.665 |                   0.661 |                    0.672 | 0.662                                     |      0.683 |
| facebook-mbart-large-50                    |   0.677 |                       0.717 |                   0.719 |                    0.721 | **0.723**                                 |      0.701 |
| gpt2                                       |   0.618 |                       0.693 |                   0.69  |                    0.673 | 0.696                                     |      0.691 |
| xlm-roberta-large                          |   0.66  |                       0.694 |                   0.718 |                    0.715 | 0.710                                     |      0.687 |

/tmp/ipykernel_25445/2331756451.py:30: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  report_table.reset_index().to_latex(latex_file, index=False)


## recall_micro

...

| language   | model_name                                 |   title |   title and first paragraph | title and 5 sentences   | title and 10 sentences   | title and first sentence each paragraph   | raw text   |
|:-----------|:-------------------------------------------|--------:|----------------------------:|:------------------------|:-------------------------|:------------------------------------------|:-----------|
| en         | AshtonIsNotHere-xlm-roberta-long-base-4096 |   0.717 |                       0.774 | 0.776                   | 0.765                    | 0.758                                     | 0.749      |
| en         | EleutherAI-gpt-neo-1.3B                    |   0.761 |                       0.794 | 0.824                   | 0.812                    | 0.810                                     | 0.796      |
| en         | EleutherAI-gpt-neo-125M                    |   0.646 |                       0.73  | 0.735                   | 0.752                    | 0.780                                     | 0.782      |
| en         | bert-base-multilingual-cased               |   0.696 |                       0.765 | 0.787                   | 0.776                    | 0.760                                     | 0.772      |
| en         | distilbert-base-multilingual-cased         |   0.684 |                       0.759 | 0.769                   | 0.735                    | 0.755                                     | 0.754      |
| en         | facebook-mbart-large-50                    |   0.778 |                       0.795 | 0.812                   | 0.804                    | 0.799                                     | 0.771      |
| en         | gpt2                                       |   0.709 |                       0.751 | 0.725                   | 0.711                    | 0.754                                     | 0.763      |
| en         | xlm-roberta-large                          |   0.759 |                       0.765 | **0.825**               | 0.804                    | 0.784                                     | 0.770      |
| fr         | AshtonIsNotHere-xlm-roberta-long-base-4096 |   0.455 |                       0.52  | 0.562                   | 0.505                    | 0.580                                     | 0.574      |
| fr         | EleutherAI-gpt-neo-1.3B                    |   0.536 |                       0.6   | 0.540                   | 0.542                    | 0.571                                     | 0.600      |
| fr         | EleutherAI-gpt-neo-125M                    |   0.324 |                       0.463 | 0.480                   | 0.440                    | 0.545                                     | **0.643**  |
| fr         | bert-base-multilingual-cased               |   0.524 |                       0.538 | 0.495                   | 0.532                    | 0.566                                     | 0.579      |
| fr         | distilbert-base-multilingual-cased         |   0.481 |                       0.51  | 0.520                   | 0.545                    | 0.615                                     | 0.575      |
| fr         | facebook-mbart-large-50                    |   0.558 |                       0.559 | 0.640                   | 0.570                    | 0.585                                     | 0.590      |
| fr         | gpt2                                       |   0.371 |                       0.443 | 0.483                   | 0.505                    | 0.500                                     | 0.602      |
| fr         | xlm-roberta-large                          |   0.505 |                       0.545 | 0.577                   | 0.547                    | 0.594                                     | 0.580      |
| ge         | AshtonIsNotHere-xlm-roberta-long-base-4096 |   0.702 |                       0.662 | 0.686                   | 0.773                    | 0.694                                     | 0.721      |
| ge         | EleutherAI-gpt-neo-1.3B                    |   0.678 |                       0.703 | 0.705                   | **0.802**                | 0.735                                     | 0.738      |
| ge         | EleutherAI-gpt-neo-125M                    |   0.533 |                       0.641 | 0.680                   | 0.651                    | 0.679                                     | 0.739      |
| ge         | bert-base-multilingual-cased               |   0.608 |                       0.677 | 0.698                   | 0.707                    | 0.633                                     | 0.697      |
| ge         | distilbert-base-multilingual-cased         |   0.658 |                       0.636 | 0.638                   | 0.699                    | 0.691                                     | 0.682      |
| ge         | facebook-mbart-large-50                    |   0.741 |                       0.667 | 0.672                   | 0.760                    | 0.715                                     | 0.752      |
| ge         | gpt2                                       |   0.559 |                       0.624 | 0.587                   | 0.652                    | 0.661                                     | 0.676      |
| ge         | xlm-roberta-large                          |   0.721 |                       0.705 | 0.711                   | 0.735                    | 0.714                                     | 0.729      |
| it         | AshtonIsNotHere-xlm-roberta-long-base-4096 |   0.705 |                       0.657 | 0.661                   | 0.735                    | 0.719                                     | 0.714      |
| it         | EleutherAI-gpt-neo-1.3B                    |   0.644 |                       0.731 | 0.756                   | 0.726                    | 0.714                                     | 0.732      |
| it         | EleutherAI-gpt-neo-125M                    |   0.392 |                       0.483 | 0.606                   | 0.586                    | 0.607                                     | 0.676      |
| it         | bert-base-multilingual-cased               |   0.569 |                       0.703 | 0.669                   | 0.692                    | 0.683                                     | 0.713      |
| it         | distilbert-base-multilingual-cased         |   0.57  |                       0.651 | 0.671                   | 0.669                    | 0.639                                     | 0.684      |
| it         | facebook-mbart-large-50                    |   0.678 |                       0.732 | 0.760                   | 0.780                    | 0.728                                     | **0.784**  |
| it         | gpt2                                       |   0.487 |                       0.578 | 0.587                   | 0.608                    | 0.628                                     | 0.609      |
| it         | xlm-roberta-large                          |   0.673 |                       0.729 | 0.765                   | 0.764                    | 0.725                                     | 0.757      |
| po         | AshtonIsNotHere-xlm-roberta-long-base-4096 |   0.604 |                       0.661 | 0.682                   | 0.750                    | 0.757                                     | 0.720      |
| po         | EleutherAI-gpt-neo-1.3B                    |   0.566 |                       0.659 | 0.725                   | 0.750                    | 0.741                                     | 0.731      |
| po         | EleutherAI-gpt-neo-125M                    |   0.544 |                       0.591 | 0.586                   | 0.622                    | 0.659                                     | 0.683      |
| po         | bert-base-multilingual-cased               |   0.633 |                       0.69  | 0.733                   | 0.724                    | 0.750                                     | 0.699      |
| po         | distilbert-base-multilingual-cased         |   0.647 |                       0.592 | 0.679                   | 0.747                    | 0.713                                     | 0.673      |
| po         | facebook-mbart-large-50                    |   0.614 |                       0.679 | 0.770                   | **0.792**                | 0.788                                     | 0.756      |
| po         | gpt2                                       |   0.559 |                       0.611 | 0.586                   | 0.684                    | 0.667                                     | 0.700      |
| po         | xlm-roberta-large                          |   0.644 |                       0.703 | 0.711                   | 0.744                    | 0.771                                     | 0.758      |
| ru         | AshtonIsNotHere-xlm-roberta-long-base-4096 |   0.5   |                       0.522 | 0.545                   | 0.561                    | 0.633                                     | 0.615      |
| ru         | EleutherAI-gpt-neo-1.3B                    |   0.576 |                       0.515 | 0.639                   | 0.632                    | 0.634                                     | 0.622      |
| ru         | EleutherAI-gpt-neo-125M                    |   0.36  |                       0.341 | 0.385                   | 0.288                    | 0.241                                     | 0.273      |
| ru         | bert-base-multilingual-cased               |   0.464 |                       0.486 | 0.580                   | 0.537                    | 0.515                                     | 0.567      |
| ru         | distilbert-base-multilingual-cased         |   0.457 |                       0.434 | 0.556                   | 0.541                    | 0.530                                     | 0.500      |
| ru         | facebook-mbart-large-50                    |   0.58  |                       0.577 | 0.655                   | 0.614                    | **0.714**                                 | 0.617      |
| ru         | gpt2                                       |   0.333 |                       0.231 | 0.190                   | 0.263                    | 0.280                                     | 0.371      |
| ru         | xlm-roberta-large                          |   0.562 |                       0.585 | 0.607                   | 0.559                    | 0.581                                     | 0.559      |

/tmp/ipykernel_25445/2331756451.py:60: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  multi_language_report_table_metric.reset_index().to_latex(latex_file, index=False)


## accuracy

| language   | model_name                                 |   title | title and first paragraph   | title and 5 sentences   | title and 10 sentences   | title and first sentence each paragraph   | raw text   |
|:-----------|:-------------------------------------------|--------:|:----------------------------|:------------------------|:-------------------------|:------------------------------------------|:-----------|
| en         | AshtonIsNotHere-xlm-roberta-long-base-4096 |   0.087 | 0.136                       | 0.126                   | 0.068                    | 0.097                                     | 0.058      |
| en         | EleutherAI-gpt-neo-1.3B                    |   0.087 | 0.087                       | **0.155**               | 0.097                    | 0.107                                     | 0.087      |
| en         | EleutherAI-gpt-neo-125M                    |   0.019 | 0.087                       | 0.058                   | 0.039                    | 0.078                                     | 0.107      |
| en         | bert-base-multilingual-cased               |   0.078 | 0.087                       | 0.117                   | 0.078                    | 0.078                                     | 0.117      |
| en         | distilbert-base-multilingual-cased         |   0.097 | 0.078                       | 0.126                   | 0.078                    | 0.117                                     | 0.117      |
| en         | facebook-mbart-large-50                    |   0.117 | 0.126                       | 0.146                   | 0.107                    | 0.126                                     | 0.126      |
| en         | gpt2                                       |   0.097 | 0.097                       | 0.117                   | 0.087                    | 0.087                                     | 0.117      |
| en         | xlm-roberta-large                          |   0.058 | 0.087                       | 0.146                   | 0.126                    | 0.107                                     | 0.107      |
| fr         | AshtonIsNotHere-xlm-roberta-long-base-4096 |   0.071 | 0.071                       | 0.071                   | 0.071                    | **0.119**                                 | 0.095      |
| fr         | EleutherAI-gpt-neo-1.3B                    |   0.024 | **0.119**                   | 0.024                   | 0.071                    | 0.024                                     | 0.071      |
| fr         | EleutherAI-gpt-neo-125M                    |   0.024 | 0.024                       | 0.024                   | 0.024                    | 0.024                                     | 0.071      |
| fr         | bert-base-multilingual-cased               |   0.048 | 0.071                       | 0.048                   | 0.071                    | 0.071                                     | 0.095      |
| fr         | distilbert-base-multilingual-cased         |   0.071 | 0.071                       | 0.095                   | 0.048                    | 0.095                                     | 0.095      |
| fr         | facebook-mbart-large-50                    |   0.024 | 0.095                       | 0.071                   | 0.048                    | 0.071                                     | 0.071      |
| fr         | gpt2                                       |   0     | 0.024                       | 0.048                   | 0.000                    | 0.048                                     | 0.048      |
| fr         | xlm-roberta-large                          |   0     | **0.119**                   | 0.071                   | 0.095                    | 0.095                                     | 0.048      |
| ge         | AshtonIsNotHere-xlm-roberta-long-base-4096 |   0.029 | 0.029                       | 0.029                   | 0.057                    | 0.057                                     | 0.029      |
| ge         | EleutherAI-gpt-neo-1.3B                    |   0     | 0.029                       | 0.000                   | 0.057                    | 0.029                                     | 0.029      |
| ge         | EleutherAI-gpt-neo-125M                    |   0     | 0.000                       | 0.029                   | 0.000                    | 0.029                                     | 0.029      |
| ge         | bert-base-multilingual-cased               |   0     | 0.029                       | 0.029                   | 0.057                    | 0.000                                     | 0.029      |
| ge         | distilbert-base-multilingual-cased         |   0.029 | 0.029                       | 0.000                   | 0.000                    | 0.029                                     | 0.000      |
| ge         | facebook-mbart-large-50                    |   0.057 | 0.029                       | 0.029                   | 0.029                    | 0.057                                     | 0.000      |
| ge         | gpt2                                       |   0     | 0.000                       | 0.029                   | **0.086**                | 0.000                                     | 0.000      |
| ge         | xlm-roberta-large                          |   0.029 | 0.057                       | 0.029                   | 0.000                    | 0.057                                     | 0.029      |
| it         | AshtonIsNotHere-xlm-roberta-long-base-4096 |   0.1   | 0.150                       | 0.167                   | 0.133                    | 0.100                                     | 0.117      |
| it         | EleutherAI-gpt-neo-1.3B                    |   0.05  | 0.117                       | 0.183                   | 0.150                    | 0.133                                     | 0.167      |
| it         | EleutherAI-gpt-neo-125M                    |   0     | 0.000                       | 0.033                   | 0.067                    | 0.033                                     | 0.117      |
| it         | bert-base-multilingual-cased               |   0.017 | 0.117                       | 0.067                   | 0.183                    | 0.117                                     | 0.133      |
| it         | distilbert-base-multilingual-cased         |   0.017 | 0.117                       | 0.117                   | 0.150                    | 0.083                                     | 0.100      |
| it         | facebook-mbart-large-50                    |   0.1   | 0.167                       | 0.150                   | 0.167                    | 0.167                                     | **0.217**  |
| it         | gpt2                                       |   0.033 | 0.033                       | 0.033                   | 0.067                    | 0.083                                     | 0.067      |
| it         | xlm-roberta-large                          |   0.1   | 0.167                       | 0.183                   | 0.133                    | 0.150                                     | 0.183      |
| po         | AshtonIsNotHere-xlm-roberta-long-base-4096 |   0.026 | 0.026                       | 0.026                   | 0.051                    | 0.051                                     | 0.051      |
| po         | EleutherAI-gpt-neo-1.3B                    |   0.026 | 0.000                       | 0.051                   | 0.051                    | 0.026                                     | 0.051      |
| po         | EleutherAI-gpt-neo-125M                    |   0     | 0.000                       | 0.051                   | 0.026                    | 0.026                                     | 0.026      |
| po         | bert-base-multilingual-cased               |   0.026 | 0.051                       | 0.051                   | 0.051                    | 0.051                                     | 0.051      |
| po         | distilbert-base-multilingual-cased         |   0.051 | 0.000                       | 0.026                   | 0.051                    | 0.051                                     | 0.026      |
| po         | facebook-mbart-large-50                    |   0.026 | 0.000                       | 0.026                   | **0.077**                | 0.026                                     | 0.026      |
| po         | gpt2                                       |   0     | 0.000                       | 0.000                   | 0.026                    | 0.026                                     | 0.026      |
| po         | xlm-roberta-large                          |   0.051 | 0.026                       | 0.026                   | 0.051                    | 0.026                                     | 0.026      |
| ru         | AshtonIsNotHere-xlm-roberta-long-base-4096 |   0.158 | 0.158                       | 0.132                   | 0.079                    | **0.263**                                 | 0.211      |
| ru         | EleutherAI-gpt-neo-1.3B                    |   0.053 | 0.105                       | 0.079                   | 0.158                    | 0.105                                     | 0.132      |
| ru         | EleutherAI-gpt-neo-125M                    |   0.026 | 0.000                       | 0.053                   | 0.000                    | 0.026                                     | 0.026      |
| ru         | bert-base-multilingual-cased               |   0.079 | 0.105                       | 0.237                   | 0.158                    | 0.158                                     | 0.237      |
| ru         | distilbert-base-multilingual-cased         |   0.105 | 0.132                       | 0.158                   | 0.158                    | 0.184                                     | 0.184      |
| ru         | facebook-mbart-large-50                    |   0.158 | 0.211                       | 0.211                   | 0.184                    | 0.184                                     | 0.184      |
| ru         | gpt2                                       |   0     | 0.000                       | 0.000                   | 0.026                    | 0.053                                     | 0.026      |
| ru         | xlm-roberta-large                          |   0.158 | 0.184                       | 0.211                   | 0.105                    | 0.184                                     | 0.158      |

/tmp/ipykernel_25445/2331756451.py:60: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  multi_language_report_table_metric.reset_index().to_latex(latex_file, index=False)
